In [1]:
import random
import cnn_builder as cbuild
import config
import copy
import glob
import importlib
import lipiodol_methods as lm
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import niftiutils.masks as masks
import numpy as np
import os
from os.path import *
import shutil
from scipy.ndimage.morphology import binary_closing, binary_opening, binary_dilation
from skimage.morphology import ball, label

from keras import backend as K
from keras.engine import Input, Model
from keras.layers import *
from keras.optimizers import Adam
from keras.utils import np_utils
%matplotlib inline

C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
importlib.reload(config)
C = config.Config()

In [3]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

In [92]:
seg = glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids"))[0]

In [ ]:
lesion_id = "07"
glob.glob(join(target_dir,lesion_id,"CT24h","*"))

In [27]:
for f in glob.glob(join(target_dir,lesion_id,"CT24h","*")):
    if "5 0" not in f:
        shutil.rmtree(f)
    else:
        shutil.move()

# MRI

### Data prep

In [4]:
# MRI
for seg in glob.glob(join(target_dir,"*","masks","liver bl.ids")):
    lesion_id = dirname(seg)[len(target_dir)+1:-len("masks")-1]
    if exists(join(C.full_img_dir, lesion_id+"_mri_art.npy")):
        continue
    
    mri_img, mri_dims = hf.nii_load(join(target_dir, lesion_id, "MRI-BL", "t1-art.nii.gz"), True, True)
    mask, _ = masks.get_mask(seg, mri_dims, mri_img.shape)

    mri_img -= mri_img.min()
    mri_img /= mri_img.std()
    
    np.save(join(C.full_img_dir, lesion_id+"_mri_art.npy"), mri_img)
    np.save(join(C.full_img_dir, lesion_id+"_mr_bl_liver_mask.npy"), mask > 128)
    
    #mri_img, mri_dims = hf.nii_load(join(target_dir, lesion_id, "MRI-BL", "t1-pre.nii.gz"), True, True)
    #mri_img -= mri_img.min()
    #mri_img /= mri_img.std()
    #np.save(join(C.full_img_dir, lesion_id+"_mri_art.npy"), mri_img)

In [13]:
data_dir = "D:\\JHU\\JHU HCC"

for path in glob.glob(join(data_dir,"*","*.nii")):
    lesion_id = basename(dirname(path))
    lesion_id = lesion_id[:lesion_id.find(" ")]
    
    mri_img, mri_dims = hf.nii_load(path)
    
    mask_paths = glob.glob(join(dirname(path),"*Segs","Liver","*.ids"))
    if len(mask_paths) != 1:
        print(path)
        continue
    mask, _ = masks.get_mask(mask_paths[0], mri_dims, mri_img.shape)

    mri_img -= mri_img.min()
    mri_img /= mri_img.std()
    
    np.save(join(C.full_img_dir, lesion_id+"_mri_art.npy"), mri_img)
    np.save(join(C.full_img_dir, lesion_id+"_mr_bl_liver_mask.npy"), mask > 128)

### Train model

In [ ]:
importlib.reload(cbuild)
model = cbuild.build_unet()
#model.summary(120)

In [19]:
hist = model.fit_generator(cbuild.train_gen_mri(), epochs=500, steps_per_epoch=5)

Epoch 1/500
5/5 [==============================] - 9s 2s/step - loss: 0.2844 - acc: 0.9317
Epoch 2/500
5/5 [==============================] - 9s 2s/step - loss: 0.2532 - acc: 0.9274
Epoch 3/500
5/5 [==============================] - 7s 1s/step - loss: 0.2142 - acc: 0.9212
Epoch 4/500
5/5 [==============================] - 7s 1s/step - loss: 0.1662 - acc: 0.9454
Epoch 5/500
5/5 [==============================] - 4s 750ms/step - loss: 0.2355 - acc: 0.9391
Epoch 6/500
5/5 [==============================] - 9s 2s/step - loss: 0.1903 - acc: 0.9350
Epoch 7/500
5/5 [==============================] - 5s 1s/step - loss: 0.2036 - acc: 0.9158
Epoch 8/500
5/5 [==============================] - 9s 2s/step - loss: 0.2046 - acc: 0.9276
Epoch 9/500
5/5 [==============================] - 6s 1s/step - loss: 0.1725 - acc: 0.9150
Epoch 10/500
5/5 [==============================] - 5s 919ms/step - loss: 0.1872 - acc: 0.9390
Epoch 11/500
5/5 [==============================] - 6s 1s/step - loss: 0.2048 - acc

5/5 [==============================] - 4s 866ms/step - loss: 0.0558 - acc: 0.9806
Epoch 89/500
5/5 [==============================] - 7s 1s/step - loss: 0.0514 - acc: 0.9820
Epoch 90/500
5/5 [==============================] - 4s 823ms/step - loss: 0.0895 - acc: 0.9666
Epoch 91/500
5/5 [==============================] - 3s 508ms/step - loss: 0.0567 - acc: 0.9806
Epoch 92/500
5/5 [==============================] - 4s 747ms/step - loss: 0.0616 - acc: 0.9785
Epoch 93/500
5/5 [==============================] - 4s 860ms/step - loss: 0.0577 - acc: 0.9774
Epoch 94/500
5/5 [==============================] - 5s 1s/step - loss: 0.0587 - acc: 0.9786
Epoch 95/500
5/5 [==============================] - 3s 518ms/step - loss: 0.0921 - acc: 0.9660
Epoch 96/500
5/5 [==============================] - 6s 1s/step - loss: 0.0702 - acc: 0.9740
Epoch 97/500
5/5 [==============================] - 3s 536ms/step - loss: 0.0769 - acc: 0.9699
Epoch 98/500
5/5 [==============================] - 6s 1s/step - loss: 0

5/5 [==============================] - 4s 847ms/step - loss: 0.0441 - acc: 0.9833
Epoch 175/500
5/5 [==============================] - 6s 1s/step - loss: 0.0403 - acc: 0.9856
Epoch 176/500
5/5 [==============================] - 3s 501ms/step - loss: 0.0584 - acc: 0.9777
Epoch 177/500
5/5 [==============================] - 5s 903ms/step - loss: 0.0641 - acc: 0.9763
Epoch 178/500
5/5 [==============================] - 4s 862ms/step - loss: 0.0458 - acc: 0.9839
Epoch 179/500
5/5 [==============================] - 4s 868ms/step - loss: 0.0557 - acc: 0.9784
Epoch 180/500
5/5 [==============================] - 8s 2s/step - loss: 0.0600 - acc: 0.9776
Epoch 181/500
5/5 [==============================] - 7s 1s/step - loss: 0.0615 - acc: 0.9749
Epoch 182/500
5/5 [==============================] - 5s 906ms/step - loss: 0.0478 - acc: 0.9830
Epoch 183/500
5/5 [==============================] - 7s 1s/step - loss: 0.0418 - acc: 0.9853
Epoch 184/500
5/5 [==============================] - 2s 493ms/step

5/5 [==============================] - 8s 2s/step - loss: 0.0582 - acc: 0.9780
Epoch 261/500
5/5 [==============================] - 3s 635ms/step - loss: 0.0828 - acc: 0.9712
Epoch 262/500
5/5 [==============================] - 3s 533ms/step - loss: 0.0460 - acc: 0.9819
Epoch 263/500
5/5 [==============================] - 4s 798ms/step - loss: 0.0436 - acc: 0.9844
Epoch 264/500
5/5 [==============================] - 5s 1s/step - loss: 0.0628 - acc: 0.9760
Epoch 265/500
5/5 [==============================] - 5s 1s/step - loss: 0.0480 - acc: 0.9819
Epoch 266/500
5/5 [==============================] - 3s 555ms/step - loss: 0.0473 - acc: 0.9819
Epoch 267/500
5/5 [==============================] - 5s 987ms/step - loss: 0.0577 - acc: 0.9801
Epoch 268/500
5/5 [==============================] - 4s 875ms/step - loss: 0.0442 - acc: 0.9827
Epoch 269/500
5/5 [==============================] - 5s 1s/step - loss: 0.0396 - acc: 0.9846
Epoch 270/500
5/5 [==============================] - 6s 1s/step - 

5/5 [==============================] - 6s 1s/step - loss: 0.0315 - acc: 0.9879
Epoch 347/500
5/5 [==============================] - 3s 649ms/step - loss: 0.0270 - acc: 0.9896
Epoch 348/500
5/5 [==============================] - 4s 869ms/step - loss: 0.0482 - acc: 0.9813
Epoch 349/500
5/5 [==============================] - 3s 587ms/step - loss: 0.0301 - acc: 0.9881
Epoch 350/500
5/5 [==============================] - 2s 459ms/step - loss: 0.0400 - acc: 0.9846
Epoch 351/500
5/5 [==============================] - 5s 920ms/step - loss: 0.0383 - acc: 0.9850
Epoch 352/500
5/5 [==============================] - 3s 543ms/step - loss: 0.0371 - acc: 0.9856
Epoch 353/500
5/5 [==============================] - 4s 815ms/step - loss: 0.0390 - acc: 0.9850
Epoch 354/500
5/5 [==============================] - 3s 575ms/step - loss: 0.0334 - acc: 0.9867
Epoch 355/500
5/5 [==============================] - 3s 547ms/step - loss: 0.0372 - acc: 0.9857
Epoch 356/500
5/5 [==============================] - 5s 9

5/5 [==============================] - 8s 2s/step - loss: 0.0283 - acc: 0.9887
Epoch 433/500
5/5 [==============================] - 3s 531ms/step - loss: 0.0329 - acc: 0.9868
Epoch 434/500
5/5 [==============================] - 5s 1s/step - loss: 0.0275 - acc: 0.9887
Epoch 435/500
5/5 [==============================] - 5s 923ms/step - loss: 0.0220 - acc: 0.9917
Epoch 436/500
5/5 [==============================] - 3s 584ms/step - loss: 0.0257 - acc: 0.9898
Epoch 437/500
5/5 [==============================] - 2s 492ms/step - loss: 0.0362 - acc: 0.9857
Epoch 438/500
5/5 [==============================] - 6s 1s/step - loss: 0.0252 - acc: 0.9900
Epoch 439/500
5/5 [==============================] - 2s 469ms/step - loss: 0.0324 - acc: 0.9873
Epoch 440/500
5/5 [==============================] - 7s 1s/step - loss: 0.0255 - acc: 0.9898
Epoch 441/500
5/5 [==============================] - 3s 579ms/step - loss: 0.0282 - acc: 0.9884
Epoch 442/500
5/5 [==============================] - 10s 2s/step -

In [20]:
model.save(join(C.model_dir, "mri_liver.hdf5"))

### Test

In [72]:
importlib.reload(lm)

<module 'lipiodol_methods' from 'C:\\Users\\Clinton\\Documents\\Lipiodol\\lipiodol_methods.py'>

In [73]:
lesion_id = "03"
mri_nii_path = join(target_dir, lesion_id, "MRI-30d", "t1-art.nii.gz")
save_path = join(target_dir, lesion_id, "masks", "mri_30d_liver")
tumor_mask_path = join(target_dir, lesion_id, "masks", "tumor 30d")
lm.seg_liver_mri_from_path(mri_nii_path, save_path, model, tumor_mask_path)

# CT

In [ ]:
train_set = []
for seg in glob.glob(join(target_dir,"*","masks","wholeliver_24hCT*.ids")):
    lesion_id = dirname(seg)[len(target_dir)+1:-len("masks")-1]
    if exists(join(C.full_img_dir, lesion_id+"_ct.npy")):
        #train_set.append([np.load(join(C.full_img_dir, lesion_id+"_ct.npy")),
        #                  np.load(join(C.full_img_dir, lesion_id+"_mask.npy")),
        #                  lesion_id])
        continue
    
    ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
    mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)

    ct_img = tr.apply_window(ct_img)
    ct_img -= np.amin(ct_img)
    
    np.save(join(C.full_img_dir, lesion_id+"_ct.npy"), ct_img)
    np.save(join(C.full_img_dir, lesion_id+"_mask.npy"), mask > 128)

In [91]:
importlib.reload(cbuild)
model = cbuild.build_ct_unet()
model.summary(120)

<module 'cnn_builder' from 'C:\\Users\\Clinton\\Documents\\Lipiodol\\cnn_builder.py'>

In [ ]:
hist = model.fit_generator(cbuild.train_gen(), epochs=100, steps_per_epoch=50)

In [38]:
model.save(join(C.model_dir, "ct17.hdf5"))

In [107]:
lesion_id = "18"
ct_img, ct_dims = hf.dcm_load(join(target_dir, lesion_id, "CT24h"), True, True)
lm.seg_liver_ct(ct_img, join(target_dir, lesion_id, "ct24_liver"), ct_dims, model)

# Misc

In [12]:
info = {}

for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        
    seg_liver(ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims)
    break